In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, ArrayType
from pyspark.sql import functions as F
import json
import requests
import importlib

ModuleNotFoundError: No module named 'notebooks'

In [14]:
spark = SparkSession\
.builder\
.appName("testSparkSession")\
.getOrCreate()

In [50]:
for temp_file_name in range(5):
    with open(f"./test_spark_data/in/{str(temp_file_name)}.json", "w+") as f:
        f.write(json.dumps(requests.get("http://localhost:4040/api/v1/applications").json()))

In [15]:
spark

In [65]:
schema = StructType()\
.add("id", "string")\
.add("name", "string")\
.add("attempts", ArrayType(StructType().add("duration", "integer")))

In [72]:
input_data = spark.readStream.json(path="./test_spark_data/in/", schema=schema)

In [81]:
w = input_data\
.select(F.explode("attempts"))\
.writeStream\
.outputMode("Append")\
.format("json")\
.option("path", "./test_spark_data/out/")\
.option("checkpointLocation", "./test_spark_data/checkpoints")\
.start()

In [83]:
w.awaitTermination()

KeyboardInterrupt: 